#Importing Colab and necessary packages

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq
!pip install emoji cleantext -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 143.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np
import joblib
from collections import Counter
from pathlib import Path
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import base64
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from functools import partial
import wandb
import matplotlib.pyplot as plt
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import re
from cleantext import clean
import emoji
from datasets import load_dataset

##Path and defining folders

In [ ]:
#google/flan-t5-base

In [ ]:
basepath = ("/content/drive/MyDrive/Applied_NLP/")
sys.path.append("/content/drive/MyDrive/Applied_NLP/Custom_Files/")
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/tmp/tmpnrcghi5u',
 '/content/drive/MyDrive/Applied_NLP/Custom_Files/']

In [ ]:
basefolder = Path(basepath)
datafolder = basefolder/"Class_7/Home_Work_5/Data/"
modelfolder = basefolder/"Class_8/Home_Work_6/Model_Checkpoint/gflan-without-emoji"
customfolder = basefolder/"Custom_Files/"

In [ ]:
from CustomPreprocessorSpacy import SpacyPreprocessor

In [ ]:
datafolder

PosixPath('/content/drive/MyDrive/Applied_NLP/Class_7/Home_Work_5/Data')

#Importing Data

In [ ]:
df_train = pd.read_csv(datafolder/"emoticons_train.csv")

In [ ]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

##Labels in the data

In [ ]:
labels = [label for label in df_train.columns if label not in ["Tweet","ID"]]
id2labels = {idx:label for idx, label in enumerate(labels)}
labels2id = {v:k for k,v in id2labels.items()}
labels2id

{'anger': 0,
 'anticipation': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'love': 5,
 'optimism': 6,
 'pessimism': 7,
 'sadness': 8,
 'surprise': 9,
 'trust': 10}

#Dropping,Preprocessing and Splitting

In [ ]:
df_train.drop("ID",axis=1,inplace=True)
df_test.drop("ID", axis=1, inplace=True)

In [ ]:
df_train["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_train["Tweet"]]
#df_train["Tweet"] = [re.sub(r'@\w+|#\w+', '', text)for text in df_train["Tweet"]]
df_test["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_test["Tweet"]]
#df_test["Tweet"] = [re.sub(r'@\w+|#\w+', '', text)for text in df_test["Tweet"]]

In [ ]:
df_train

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7719,@BadHombreNPS @SecretaryPerry If this didn't m...,1,0,1,0,0,0,0,0,0,0,0
7720,Excited to watch #stateoforigin tonight! Come ...,0,0,0,0,1,0,1,0,0,0,0
7721,"Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi...",1,0,1,0,0,0,0,0,1,0,0
7722,#ThingsIveLearned The wise #shepherd never tru...,0,0,0,0,0,0,0,0,0,0,0


##Spacy Preprocessing

In [ ]:
sp = SpacyPreprocessor(model='en_core_web_sm', batch_size=64, lemmatize=True, lower=True,
                                    remove_stop=True, remove_punct=True, remove_email=True,
                                    remove_url=True, remove_num=False, stemming=False,
                                    add_user_mention_prefix=False, remove_hashtag_prefix=True, basic_clean_only=False)

In [ ]:
X_train_total = df_train["Tweet"].tolist()

In [ ]:
X_test = df_test["Tweet"].tolist()

In [ ]:
X_train_total

["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'Whatever you decide to do make sure it makes you #happy.',
 "@Max_Kellerman  it also helps that the majority of NFL coaching is inept. Some of Bill O'Brien's play calling was wow, ! #GOPATS",
 "Accept the challenges so that you can literally even feel the exhilaration of victory.' -- George S. Patton ",
 "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs",
 "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu",
 'Do you think humans have the sense for recognizing impending doom? ',
 "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight",
 "it's pretty depressing when u hit pan on ur favourite highlighter",
 '@BossUpJaee but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .',
 '

In [ ]:
y_test = df_train

In [ ]:
y_train_total = df_train.drop("Tweet",axis=1).to_numpy().astype(float)
y_test = df_test.drop("Tweet",axis=1).to_numpy()

In [ ]:
y_train_total

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y_test = np.where(y_test == "NONE",0,y_test).astype(float)

##Train-Valid Split

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_total,y_train_total,test_size=0.3, random_state=42, shuffle=True)

##Spacy preprocessor

In [ ]:
X_train_preprocessed = sp.fit_transform(X_train)
X_valid_preprocessed = sp.transform(X_valid)
X_test_preprocessed = sp.transform(X_test)

/content/drive/MyDrive/Applied_NLP/Custom_Files/CustomPreprocessorSpacy.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


##Data Dictionaries

In [ ]:
X_train_preprocessed

['@kmunie7 @karanesch helluva lot animated actual game >.<',
 "despite sharing o'neill article sadden gazza case fury sentence remark",
 '@mtailor 20 well tailor tailor know #navy vs. #blue $ 500 4 week #innacurate suit',
 'earth projection good happy true soul poisonous fucking snake #patronus #adder #offende',
 'patti sad stamp run sofa plenty love affection usual',
 '@wwe live event brock vs orton 2 saturday wwe network need #wwe @randyorton @heymanhustle #revenge',
 '@riotzxvl papercut sting stub ur toe like 10 sec',
 '@smendler 2   watch ppl fill w/ impotent rage time lead',
 '@tbergg1 good rn go to horrid week',
 'watch amazing broadcast @kana_blender   #musically',
 '@twojacksdetail @bluelivesmtr rare officer shoot regard want conscience #incite',
 '@bennymcnugget sad upset idk',
 '@orbsofjoy plan date like date u find pleasing smth fuckign\\n\\n10/10 child grow',
 'ya girl',
 '@itejasmehta @qtalker pleasure',
 'incomplete state tails possess immense physical strength remarkably

In [ ]:
trainset = Dataset.from_dict({
    'texts': X_train_preprocessed,
    'labels': y_train
})

validset = Dataset.from_dict({
    'texts': X_valid_preprocessed,
    'labels': y_valid
})

testset = Dataset.from_dict({
    'texts': X_test_preprocessed,
    'labels': y_test
})

In [ ]:
trainset[0:5]

{'texts': ['@kmunie7 @karanesch helluva lot animated actual game >.<',
  "despite sharing o'neill article sadden gazza case fury sentence remark",
  '@mtailor 20 well tailor tailor know #navy vs. #blue $ 500 4 week #innacurate suit',
  'earth projection good happy true soul poisonous fucking snake #patronus #adder #offende',
  'patti sad stamp run sofa plenty love affection usual'],
 'labels': [[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]]}

In [ ]:
testset[0:5]

{'texts': ['@adnan__786 @asyounotwish not worry indian army way dispatch terrorists hell',
  'academy sciences eschew normally sober tone scientific paper call massive loss wildlife biological annihilation',
  'blow opportunity #mad',
  'time 2 week 30',
  '#deppression real partner w/ #depresse people truly not understand depth affect add #anxiety make bad'],
 'labels': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]}

In [ ]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

#Distil-bert as Checkpoint

In [ ]:
checkpoint_google_flan = "google/flan-t5-base"
tokenizer_google_flan = AutoTokenizer.from_pretrained(checkpoint_google_flan)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

##Tokenize for distil-bert



In [ ]:
def tokenize_fn_googleflan(batch):
  return tokenizer_google_flan(text = batch["texts"], truncation=True, padding="max_length",return_tensors = "pt", max_length = 128)

In [ ]:
tokenized_dataset_train_gflan = trainset.map(tokenize_fn_googleflan,batched=True)
tokenized_dataset_valid_gflan = validset.map(tokenize_fn_googleflan,batched=True)
tokenized_dataset_test_gflan = testset.map(tokenize_fn_googleflan,batched=True)

Map:   0%|          | 0/5406 [00:00<?, ? examples/s]

Map:   0%|          | 0/2318 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset_train_gflan

Dataset({
    features: ['texts', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [ ]:
tokenized_dataset_train_gflan = tokenized_dataset_train_gflan.remove_columns(["texts"])
tokenized_dataset_valid_gflan = tokenized_dataset_valid_gflan.remove_columns(["texts"])
tokenized_dataset_test_gflan = tokenized_dataset_test_gflan.remove_columns(["texts"])

In [ ]:
tokenized_dataset_train_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [ ]:
tokenized_dataset_train_gflan.set_format(type='torch')
tokenized_dataset_valid_gflan.set_format(type='torch')
tokenized_dataset_test_gflan.set_format(type='torch')

In [ ]:
tokenized_dataset_train_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [ ]:
tokenized_dataset_test_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3259
})

##Model for distil-bert

In [ ]:
model_gflan = AutoModelForSequenceClassification.from_pretrained(checkpoint_google_flan,num_labels = 11,problem_type="multi_label_classification",id2label=id2labels,label2id=labels2id)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate
from transformers import EvalPrediction
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

##Metrics labelling functions

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,'Accuracy': accuracy}
    return metrics

def compute_metrics_1(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result

##Training Args

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=0.00002,
    optim='adamw_torch',
    remove_unused_columns=False,
    output_dir=str(modelfolder),
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_strategy='steps',
    logging_steps=10,
    report_to='wandb',
    run_name='emoticon-gflan-without-emoji',
)

##G-FLAN Trainer

In [ ]:
trainer_gflan = Trainer(
    model=model_gflan,
    args=training_args,
    train_dataset = tokenized_dataset_train_gflan,
    eval_dataset = tokenized_dataset_valid_gflan,
    compute_metrics=compute_metrics_1,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
!wandb login 5b0f2851a77f589c556f16598a0b181257a541e9

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%env WANDB_PROJECT = nlp_course_spring_2024-emtoicons-analysis-hf-trainer

env: WANDB_PROJECT=nlp_course_spring_2024-emtoicons-analysis-hf-trainer


##Training

In [ ]:
trainer_gflan.train()

wandb: Currently logged in as: abhishekmoguluri (abhishek-moguluri). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1,Accuracy
100,0.470400,0.476398,0.000000,0.023727
200,0.472200,0.472004,0.045535,0.026316
300,0.453400,0.462908,0.177990,0.052632
400,0.443700,0.441249,0.199018,0.056083
500,0.393600,0.412544,0.481925,0.150129
600,0.415800,0.394578,0.522487,0.166523
700,0.414600,0.385064,0.538942,0.173857
800,0.398600,0.385821,0.549713,0.180328
900,0.383600,0.380815,0.562847,0.186368
1000,0.368000,0.378987,0.574134,0.186368


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=3380, training_loss=0.37220350228823146, metrics={'train_runtime': 1200.9602, 'train_samples_per_second': 22.507, 'train_steps_per_second': 2.814, 'total_flos': 4127468669130240.0, 'train_loss': 0.37220350228823146, 'epoch': 5.0})

In [ ]:
best_model_gflan_checkpoint_step = trainer_gflan.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_gflan_checkpoint_step}.")

The best model was saved at step 2800.


In [ ]:
wandb.finish()

eval/Accuracy,▁▁▂▂▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████
eval/f1,▁▂▃▃▆▇▇▇▇▇▇▇▇████████████████████
eval/loss,██▇▆▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▅▂▂▄▃▂▇▂▇▆▄▂▂▂▄▄▄▁▃▁▁▁▂▃▂▄▂▇▂█▂▄▆
eval/samples_per_second,▄▇▇▄▆▇▂▇▂▃▅▇▆▇▅▅▅█▆▇██▇▆▇▅▇▂▇▁▇▅▃
eval/steps_per_second,▄▇▇▄▆▇▂▇▂▃▅▇▆▇▅▅▅█▆▇██▇▆▇▅▇▂▇▁▇▅▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▁▂▁▂▁▂▂▂▂▂▂▂▂▂▂▁▂▁▃▃▂█▃▂▂▂▃▂▁▁▂▂▂▂▂▂▂▁▂
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▅▅▅▅▄▃▄▃▄▄▃▃▃▂▃▃▃▃▃▃▃▃▃▃▂▃▃▃▁▂▁▂▂▂▃▁▂▂▂


##Save the model

In [ ]:
model_gflan = str(modelfolder/f'checkpoint-{best_model_gflan_checkpoint_step}')
model_gflan

'/content/drive/MyDrive/Applied_NLP/Class_8/Home_Work_6/Model_Checkpoint/gflan-without-emoji/checkpoint-2800'

In [ ]:
model_gflan = AutoModelForSequenceClassification.from_pretrained(model_gflan)

##Validation set

In [ ]:
training_args_gflan_test = TrainingArguments(
    output_dir=str(modelfolder),
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to='wandb',
    run_name='emoticon_hf_trainer',
)

In [ ]:
trainer_gflan_test = Trainer(
    model=model_gflan,
    args=training_args_gflan_test,
    eval_dataset=tokenized_dataset_test_gflan,
    compute_metrics=compute_metrics_1,)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
gflan_test_results = trainer_gflan_test.evaluate()

In [ ]:
gflan_test_results

{'eval_loss': 0.4407970607280731,
 'eval_f1': 0.0,
 'eval_Accuracy': 0.061061675360540045,
 'eval_runtime': 18.5598,
 'eval_samples_per_second': 175.595,
 'eval_steps_per_second': 21.983}

In [ ]:
wandb.log({"test_accuracy": gflan_test_results["eval_Accuracy"], "test_loss": gflan_test_results["eval_loss"], "test_f1": gflan_test_results["eval_f1"]})

In [ ]:
gflan_test_output = trainer_gflan_test.predict(tokenized_dataset_test_gflan)
gflan_test_output

PredictionOutput(predictions=(array([[ 2.0123765, -3.2230709,  1.6609501, ..., -1.3822589, -3.6724067,
        -4.740641 ],
       [ 1.9232371, -2.5105891,  2.0979514, ..., -0.4986269, -3.5953326,
        -4.7940273],
       [ 1.0460323, -3.4778948,  1.1197634, ...,  1.0848895, -3.9105523,
        -5.311583 ],
       ...,
       [ 3.5551515, -2.931797 ,  2.787439 , ..., -1.5179436, -3.6488569,
        -5.030208 ],
       [-3.692576 , -2.4760315, -4.1755147, ..., -3.4740345, -2.8498073,
        -1.8486023],
       [-3.2145498, -1.6167115, -3.4656684, ..., -3.6213875, -3.1384873,
        -2.669773 ]], dtype=float32), array([[[-0.05147157,  0.16170296, -0.08810655, ...,  0.06884906,
          0.17103873,  0.01931249],
        [ 0.00651529,  0.1722504 , -0.24501882, ...,  0.03203593,
          0.11285141, -0.11174151],
        [-0.09963232,  0.14270651, -0.11770652, ..., -0.02358586,
          0.00633788,  0.0102059 ],
        ...,
        [-0.1650259 ,  0.3407169 , -0.16848382, ...,  0.15

In [ ]:
gflan_test_preds = gflan_test_output.predictions

In [ ]:
gflan_test_preds = torch.tensor(gflan_test_preds[0])

##Predictions to Probablities

In [ ]:
def coversion(predictions, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    results = np.zeros(probs.shape)
    results[np.where(probs >= threshold)] = 1
    return results

In [ ]:
gflan_test_output_final = coversion(gflan_test_preds)
(gflan_test_output_final)

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

In [ ]:
df_test.columns

Index(['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [ ]:
df_test[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = gflan_test_output_final

In [ ]:
df_test.drop(["Tweet"],axis =1, inplace = True)

In [ ]:
df_test.to_csv(basefolder/"Class_8/Home_Work_6/Submissions/gdlan-test-2.csv")

##Inference

In [ ]:
mapped_labels =[]
for prediction_row in gflan_test_output_final:
    mapped_row = [label for label, pred in zip(labels, prediction_row) if pred]
    mapped_labels.append(mapped_row)

In [ ]:
from pprint import pprint
pprint(mapped_labels[:10])

[['anger', 'disgust', 'fear'],
 ['anger', 'disgust', 'sadness'],
 ['anger', 'disgust', 'fear', 'sadness'],
 ['joy', 'sadness'],
 ['fear', 'pessimism', 'sadness'],
 ['anger', 'disgust', 'fear'],
 ['joy', 'optimism'],
 ['joy', 'love', 'optimism'],
 ['joy', 'love', 'optimism'],
 ['fear', 'pessimism', 'sadness']]
